In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests
!pip3 install pandas
!pip3 install numpy

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [15]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={"class":'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).text.replace('$', '').strip()
    except AttributeError:
        price = ""
    
    return price

# Function to extract Product Rating
def get_review_count(soup):
    try:
        ratings_count = soup.find("span", attrs={"id":'acrCustomerReviewText'}).text.replace('ratings', '').strip()
    except AttributeError:
        ratings_count = ""
    
    return ratings_count

def get_author(soup):
    try:
        author_span = soup.find("span", attrs={"class": "author notFaded"})
        author_name = author_span.find("a", attrs={"class": "a-link-normal"}).get_text(strip=True)
    except AttributeError:
        author_name = ""
    
    return author_name

def get_pages(soup):
    try:
        pages_text = soup.select_one("div.a-section.a-spacing-none.a-text-center.rpi-attribute-value > span").text.replace('pages', '').strip()
    except AttributeError:
        pages_text = ""
    
    return pages_text

# Function to extract Number of User Reviews
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={"class":'a-icon-alt'}).text.replace('out of 5 stars', '').strip()
    except AttributeError:
        rating = ""
    
    return rating





In [ ]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=CLIMATE+ACTION+BOOKS&crid=14N78C5AIN3LF&sprefix=climate+action+books%2Caps%2C198&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "author":[], "price":[], "n_pages":[],"avg_rating":[], "n_reviews":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get(f"https://www.amazon.com{link}", headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")


        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['author'].append(get_author(new_soup))
        d['price'].append(get_price(new_soup))
        d['n_pages'].append(get_pages(new_soup))
        d['avg_rating'].append(get_rating(new_soup))
        d['n_reviews'].append(get_review_count(new_soup))

    
# Replace empty titles with NaN and drop rows with NaN titles
amazon_df['title'] = amazon_df['title'].replace('', np.nan)
amazon_df = amazon_df.dropna(subset=['title'])

# Check if the DataFrame is not empty before saving
if not amazon_df.empty:
    # Save the DataFrame to a CSV file
    file_path = "amazon_data.csv"
    amazon_df.to_csv(file_path, header=True, index=False)
    print(f"File successfully created at: {file_path}")
else:
    print("No data available to save.")

In [18]:
amazon_df

,title,author,price,n_pages,avg_rating,n_reviews
0,The Climate Action Handbook: A Visual Guide to...,Heidi Roop,16.60,272,4.4,25 ratings
1,Climate Action: What Happened and What We Can Do,Seymour Simon,18.99,6 - 10 years,3.8,5 ratings
2,Climate Action Planning: A Guide to Creating L...,Michael R. Boswell PhD,40.51,1610919637,4.7,13 ratings
3,Climate Action for Busy People,Cate Mingoya-LaFortune,27.00,216,5.0,5 ratings
4,Climate Information for Public Health Action (...,Madeleine C. Thomson,,978-1138069640,3.7,11 ratings
5,Saving Ourselves: From Climate Shocks to Clima...,Dana R. Fisher,17.65,224,4.6,12 ratings
6,Climate Change Simplified: A Comprehensive Gui...,Alexa Ingram,15.99,186,4.8,88 ratings
7,Speed & Scale: An Action Plan for Solving Our ...,John Doerr,,11 hours and 58 minutes,4.6,589 ratings
8,The Storm: A Climate Fiction Thriller (Exposé ...,James H. Speer,,284,4.1,16 ratings
9,Global Climate Change: Turning Knowledge Into ...,David Kitchen,50.39,0367704951,Previous slide of product details,
